# Ansbile playbook

## 1. Simple playbook

### 1.1. Run playbook with current user

In [ ]:
ansible-playbook ping.yml

### 1.2. Run playbook with become user

In [ ]:
ansible-playbook ping_root.yml --vault-id vault-id -e "@password.yml"

### 1.3. Show ansible variables

#### 1.3.1. Show simple variables

In [ ]:
ENV_VAR="HELLO WORLD" ansible-playbook vars.yml

#### 1.3.2. Show all variables

In [ ]:
SHOW_ALL=true ENV_VAR="HELLO WORLD" ansible-playbook vars.yml

### 1.3. Conditions

#### 1.3.1. Basic conditionals with `when`

In [ ]:
ansible-playbook conditions.yml --tags T1

#### 1.3.2. Conditionals based on `ansible_facts`

In [ ]:
ansible-playbook conditions.yml --tags T2